In [1]:
import pandas as pd
import os

def load_coding_table():
    """
    Load the CODING sheet from the codebook.xlsx file into a pandas DataFrame.
    
    Returns:
        pandas.DataFrame: The coding table from the CODING sheet
    """
    # Define the path to the Excel file
    excel_path = os.path.join("..", "Data", "codebook.xlsx")
    
    try:
        # Read the CODING sheet from the Excel file
        coding_table = pd.read_excel(excel_path, sheet_name="CODING")
        
        print(f"Successfully loaded CODING table with {len(coding_table)} rows and {len(coding_table.columns)} columns")
        print(f"Columns: {list(coding_table.columns)}")
        
        return coding_table
    
    except FileNotFoundError:
        print(f"Error: File not found at {excel_path}")
        return None
    except Exception as e:
        print(f"Error reading Excel file: {str(e)}")
        return None

# Alternative function if you want to specify a custom path
def load_coding_table_from_path(file_path):
    """
    Load the CODING sheet from a specified Excel file path.
    
    Args:
        file_path (str): Path to the Excel file
        
    Returns:
        pandas.DataFrame: The coding table from the CODING sheet
    """
    try:
        # Read the CODING sheet from the Excel file
        coding_table = pd.read_excel(file_path, sheet_name="CODING")
        
        print(f"Successfully loaded CODING table with {len(coding_table)} rows and {len(coding_table.columns)} columns")
        print(f"Columns: {list(coding_table.columns)}")
        
        return coding_table
    
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except Exception as e:
        print(f"Error reading Excel file: {str(e)}")
        return None

# Usage examples:
# Load using relative path
coding_data = load_coding_table()

Successfully loaded CODING table with 8847 rows and 59 columns
Columns: ['ResponseId', 'Q37_feedback_on_work', 'Q38_1_feedback_helpful_AI', 'Q38_2_feedback_helpful_teacher', 'Q39_1_feedback_trust_AI', 'Q39_2_feedback_trust_teacher', 'Please describe any differences in how it impacted your learning (comparing GenAI and your teacher)?', 'Good Quote', 'Actor', 'Comparator', 'Characteristic', 'Actor.1', 'Comparator.1', 'Characteristic.1', 'Actor.2', 'Comparator.2', 'Characteristic.2', 'Actor.3', 'Comparator.3', 'Characteristic.3', 'Actor.4', 'Comparator.4', 'Characteristic.4', 'Actor.5', 'Comparator.5', 'Characteristic.5', 'Were there any differences in how the feedback made you feel (comparing GenAI and your teacher)?', 'Good Quote.1', 'Actor.6', 'Comparator.6', 'Characteristic.6', 'Actor.7', 'Comparator.7', 'Characteristic.7', 'Actor.8', 'Comparator.8', 'Characteristic.8', 'Actor.9', 'Comparator.9', 'Characteristic.9', 'Describe any other differences between the feedback you received fro

In [4]:
def filter_coding_data(coding_data):
    """
    Filter the coding data to keep only rows that have data in the specified columns.
    
    Args:
        coding_data (pandas.DataFrame): The original coding data
        
    Returns:
        pandas.DataFrame: Filtered data containing only rows with data in columns 6, 26, 40, or 54
    """
    # Get the column names for the specified columns
    col_6 = coding_data.columns[6]  # "Please describe any differences in how it impacted your learning..."
    col_26 = coding_data.columns[26]  # "Were there any differences in how the feedback made you feel..."
    col_40 = coding_data.columns[40]  # "Describe any other differences between the feedback..."
    col_54 = coding_data.columns[54]  # "Why didn't you use GenAI for feedback on your work?"
    
    # Filter rows that have non-null data in any of these columns
    filtered_data = coding_data[
        coding_data[col_6].notna() | 
        coding_data[col_26].notna() | 
        coding_data[col_40].notna() | 
        coding_data[col_54].notna()
    ].copy()
    
    print(f"Original data: {len(coding_data)} rows")
    print(f"Filtered data: {len(filtered_data)} rows")
    print(f"Kept {len(filtered_data)} rows with data in columns 6, 26, 40, or 54")
    
    # Show breakdown of which columns have data
    print(f"\nBreakdown of non-null values in target columns:")
    print(f"Column 6 ({col_6}): {filtered_data[col_6].notna().sum()} rows")
    print(f"Column 26 ({col_26}): {filtered_data[col_26].notna().sum()} rows")
    print(f"Column 40 ({col_40}): {filtered_data[col_40].notna().sum()} rows")
    print(f"Column 54 ({col_54}): {filtered_data[col_54].notna().sum()} rows")
    
    return filtered_data

filtered_coding = filter_coding_data(coding_data)

Original data: 8847 rows
Filtered data: 5448 rows
Kept 5448 rows with data in columns 6, 26, 40, or 54

Breakdown of non-null values in target columns:
Column 6 (Please describe any differences in how it impacted your learning (comparing GenAI and your teacher)?): 2420 rows
Column 26 (Were there any differences in how the feedback made you feel (comparing GenAI and your teacher)?): 2356 rows
Column 40 (Describe any other differences between the feedback you received from GenAI and your teacher.): 1808 rows
Column 54 (Why didn’t you use GenAI for feedback on your work?): 2989 rows


In [10]:
filtered_coding.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5448 entries, 1 to 8845
Data columns (total 59 columns):
 #   Column                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                --------------  -----  
 0   ResponseId                                                                                            5448 non-null   object 
 1   Q37_feedback_on_work                                                                                  5447 non-null   float64
 2   Q38_1_feedback_helpful_AI                                                                             2458 non-null   float64
 3   Q38_2_feedback_helpful_teacher                                                                        2458 non-null   float64
 4   Q39_1_feedback_trust_AI                                                                               2458 

In [9]:
def save_survey_responses_to_files(filtered_data):
    """
    Save each row's responses to individual text files named by ResponseId.
    Only includes columns 6, 26, 40, and 54 with non-null values.
    
    Args:
        filtered_data (pandas.DataFrame): The filtered coding data
    """
    # Create the survey directory if it doesn't exist
    survey_dir = "./survey"
    if not os.path.exists(survey_dir):
        os.makedirs(survey_dir)
        print(f"Created directory: {survey_dir}")
    
    # Define the target columns and their indices
    target_columns = [
        (6, "Please describe any differences in how it impacted your learning (comparing GenAI and your teacher)?"),
        (26, "Were there any differences in how the feedback made you feel (comparing GenAI and your teacher)?"),
        (40, "Describe any other differences between the feedback you received from GenAI and your teacher."),
        (54, "Why didn't you use GenAI for feedback on your work?")
    ]
    
    files_created = 0
    
    # Process each row
    for index, row in filtered_data.iterrows():
        response_id = row['ResponseId']
        
        # Skip if ResponseId is null or empty
        if pd.isna(response_id) or str(response_id).strip() == '':
            continue
        
        # Create filename (clean the ResponseId to make it safe for filenames)
        safe_filename = str(response_id).replace('/', '_').replace('\\', '_').replace(':', '_')
        file_path = os.path.join(survey_dir, f"{safe_filename}.txt")
        
        # Build the content for this file
        content_parts = []
        
        for col_idx, question in target_columns:
            answer = row.iloc[col_idx]
            
            # Only include if the answer is not null and not empty
            if pd.notna(answer) and str(answer).strip() != '':
                content_parts.append(f"Question:\n{question}\n\nAnswer:\n{answer}\n\n")
        
        # Only create file if there's content
        if content_parts:
            try:
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write(''.join(content_parts))
                files_created += 1
                
                if files_created <= 5:  # Show first 5 files for verification
                    print(f"Created: {file_path}")
                    
            except Exception as e:
                print(f"Error creating file for ResponseId {response_id}: {str(e)}")
    
    print(f"\nTotal files created: {files_created}")
    print(f"Files saved in directory: {survey_dir}")
    
save_survey_responses_to_files(filtered_coding)

Created directory: ./survey
Created: ./survey\R_4Tt0Hd9yrbZWVwr.txt
Created: ./survey\R_4QU6UODvFFxSRYR.txt
Created: ./survey\R_4dh8NUEF94hp8Ey.txt
Created: ./survey\R_42nqrlSvWlTyD9x.txt
Created: ./survey\R_4ksNFrS9UmgYiAd.txt

Total files created: 5448
Files saved in directory: ./survey
